In [9]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
num_samples = 44100

In [11]:
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

In [12]:
!cat 01_simple.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter { }
signal OuterCounter { reset: ResetOuter}

loop Loop {
	ports: [		
	]
	blocks: [
		signal Counter {}
		switch CounterReset {}
	]
	streams: [
		Accum + 2 >> Accum;
		Counter + 1 >> Counter;
		[Counter, 10] >> Equal() >> CounterReset;
	]
	terminateWhen: CounterReset
}


[OuterCounter, 20] >> Equal() >> ResetOuter >> Loop(); 
OuterCounter + 1 >> OuterCounter;
Accum >> AudioOut[1];



In [13]:
out_text = ''

outercounter = 0
counter = 0
accum = 0

for val1, val2 in zip(input1, input2):
    if outercounter == 20:
        for i in range(10):
            accum += 2
        outercounter = 0
    outercounter += 1
    out_text += str(accum) + "\n0.0\n"

out_file = open("01_simple.expected", "w")
out_file.write(out_text[:-1])
pass

In [14]:
!cat 02_loop_in_module.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter { }
signal OuterCounter { reset: ResetOuter}

module LoopModule {
	ports: [
		mainOutputPort OutputPort { block: Output }
	]
	blocks:  [
		signal Accumulator { default: 0}
		loop Accumulate {
			blocks: [
				signal Counter {default: 0}
				switch CounterReset {}
			]
			streams: [
				Accumulator + 1.5 >> Accumulator;
				Counter + 1 >> Counter;
				[Counter, 10] >> Equal() >> CounterReset;
			]
			terminateWhen: CounterReset
		}
	]
	streams: [
		on >> Accumulate();
		Accumulator >> Output;
	]
}

LoopModule() >> AudioOut[1];



In [15]:
out_text = ''

counter = 0
accum = 0

for val1, val2 in zip(input1, input2):
    for i in range(10):
        accum += 1.5
    out_text += str(accum) + "\n0.0\n"

out_file = open("02_loop_in_module.expected", "w")
out_file.write(out_text[:-1])
pass

In [16]:
!cat 03_list_input_in_module.stride

use DesktopAudio version 1.0

signal Accum {}
switch ResetOuter { }
signal OuterCounter { reset: ResetOuter}

module Sum {
	ports: [
		mainOutputPort OutputPort { block: Output }
		mainInputPort InputPort { block: Input }
	]
	blocks:  [
		signal Input[3] { domain: Output.domain }
		signal Accumulator { default: 0 reset: ResetAccumulator}
		switch ResetAccumulator {}
		signal Counter {default: 1}
		loop Accumulate {
			ports: [	
				mainOutputPort OutputPort { block: Output }
			]
			blocks: [
				switch CounterReset {}
			]
			streams: [
				Accumulator + Input[Counter] >> Accumulator;
				Counter + 1 >> Counter;
				[Counter, 3] >> Greater() >> CounterReset;
			]
			terminateWhen: CounterReset
		}
	]
	streams: [
		on >> Accumulate();
	]
}

[AudioIn[1], AudioIn[2], 7] >> Sum() >> Value;

